In [45]:
import gurobipy as gb 
from gurobipy import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [46]:
#Supply Data
supply_input = pd.read_excel('code_input_supply.xlsx')
supply_input['Capacity'] = supply_input['Capacity']*24*30

plant_names = supply_input['Name'].tolist()
capacity = supply_input['Capacity'].tolist()
year_of_construction = supply_input['Year'].tolist()

In [47]:
#Distance Data
distance_input = pd.read_excel('code_input_distance.xlsx')
pivot_table = distance_input.pivot(index='station_name', columns='muni_name', values='normalized_distance')
distance_matrix = pivot_table.values.tolist()

In [48]:
#Data Prep for Demand Data

demand_base = pd.read_excel('code_input_demand.xlsx')

#Demand Projection for 2024 based on 2020 demand - 11% growth from 2020 to 2024

#For the Municipalities which have data in 2020

demand_base2020 = demand_base[demand_base['Year'] == 2020]
grouped_df = demand_base2020.groupby(['Municipality', 'Year', 'Month'])
sum_consumption = grouped_df['Total Consumption (MWh)'].sum()
sum_consumption = sum_consumption.reset_index()
sum_consumption = sum_consumption.rename(columns={'Total Consumption (MWh)': 'Demand'})
average_demand = sum_consumption.groupby(['Municipality', 'Month'])['Demand'].mean()
average_demand = average_demand.reset_index()
average_demand = average_demand.groupby('Municipality').filter(lambda x: len(x) == 12)

#For other municipalities we will use their average demand

demand_base_others = demand_base[~demand_base['Municipality'].isin(average_demand['Municipality'])]
demand_base_others = demand_base_others[demand_base_others['Year'] != 2023]
grouped_df2 = demand_base_others.groupby(['Municipality', 'Year', 'Month'])
sum_consumption2 = grouped_df2['Total Consumption (MWh)'].sum()
sum_consumption2 = sum_consumption2.reset_index()
sum_consumption2 = sum_consumption2.rename(columns={'Total Consumption (MWh)': 'Demand'})
average_demand2 = sum_consumption2.groupby(['Municipality', 'Month'])['Demand'].mean()
average_demand2 = average_demand2.reset_index()

expected_demand = pd.concat([average_demand, average_demand2])
expected_demand = expected_demand.sort_values(by=['Municipality', 'Month'])
expected_demand['Demand'] = expected_demand['Demand']*1.11
expected_demand['Demand'] = np.ceil(expected_demand['Demand'])

expected_demand.sort_values(by=['Municipality', 'Month'], inplace=True)
city_names = expected_demand['Municipality'].unique().tolist()

demand_matrix = expected_demand.pivot(index='Municipality', columns='Month', values='Demand')
demand_matrix = demand_matrix.values.tolist()

In [49]:
prob = gb.Model("Power Grid Optimization - Model 1")
prob.params.LogToConsole = 0

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct','Nov','Dec']

I = len(plant_names)
J = len(city_names)
K = len(month_names)

In [50]:
# Decision Variables

x = prob.addVars(I, J, K, vtype=GRB.CONTINUOUS, name=[f'Power from {i} to {j} in {k}' for i in plant_names for j in city_names for k in month_names])
p = prob.addVars(I, K, vtype=GRB.CONTINUOUS, name=[f'Power generated at {i} in {k}' for i in plant_names for k in month_names])

In [51]:
#Objective function Formulation

#1. Minimize the total cost of power generation

production_cost_val = 3.456 #Cost of power generation per MWh

cost_power_generation = sum(production_cost_val*p[i,k] for i in range(I) for k in range(K))

#2. Minimize the total loss from transmission across the network

dist_loss = sum(x[i,j,k]*(0.08+0.07*distance_matrix[i][j]) for i in range(I) for j in range(J) for k in range(K))
cost_dist_loss = production_cost_val*dist_loss

total_cost_factor = cost_power_generation + cost_dist_loss

#4. Depreciation factor for operating power plants - Not an objective of the model, but added to prioritize newer plants

deprec_factor = sum((2024 - year_of_construction[i])*0.015*p[i,k] for i in range(I) for k in range(K))

prob.setObjectiveN(total_cost_factor, index=0, priority=2)
prob.setObjectiveN(deprec_factor, index=1, priority=1)

prob.modelSense = GRB.MINIMIZE

#prob.setObjective(total_cost_factor, GRB.MINIMIZE)

print("Objective Function Created")

Objective Function Created


In [52]:
#Constraints

constr_count = 0

#1. Power plants cannot run at more than 80% capacity and only if it is turned on
prob.addConstrs(p[i,k] <= 0.8*capacity[i] for i in range(I) for k in range(K))
constr_count += I*K

#2. Total Production must be atleast 20% higher than the demand in each month
prob.addConstrs(sum(p[i,k] for i in range(I)) >= 1.2*sum(demand_matrix[j][k] for j in range(J)) for k in range(K))
constr_count += K

#3. Cannot distribute more than produced in each month
prob.addConstrs(sum(x[i,j,k] for j in range(J)) <= p[i,k] for i in range(I) for k in range(K))
constr_count += I*K

#4. Distribution - Losses must be equal to demand
prob.addConstrs(sum((x[i,j,k] - (x[i,j,k]*(0.08+0.07*distance_matrix[i][j]))) for i in range(I)) == demand_matrix[j][k] for j in range(J) for k in range(K))
constr_count += J*K

#5. Avoid irregular changes in power generation across months
prob.addConstrs(p[i,k] - p[i,k-1] <= 0.2*capacity[i] for i in range(I) for k in range(1,K))
prob.addConstrs(p[i,k] - p[i,k-1] >= -0.2*capacity[i] for i in range(I) for k in range(1,K))
constr_count += 2*I*(K-1)

print(f"{constr_count} Constraints added to the model")


7412 Constraints added to the model


In [53]:
prob.optimize()
if prob.status == GRB.OPTIMAL:
    print("Optimal Solution Found")
else:
    print("No Optimal Solution Found")

Optimal Solution Found


In [54]:
# Monthly Power Generation Plan for each Plant

monthly_operation_plan = pd.DataFrame(columns=['Plant', 'Month', 'Power Generated (MWh)'])

for i in range(I):
    for k in range(K):
        monthly_operation_plan = pd.concat([monthly_operation_plan, pd.DataFrame({'Plant': plant_names[i], 'Month': month_names[k], 'Power Generated (MWh)': p[i,k].x}, index=[0])])

monthly_operation_plan['Power Generated (MWh)'] = np.ceil(monthly_operation_plan['Power Generated (MWh)'])

monthly_operating_capacity = monthly_operation_plan.copy()
monthly_operating_capacity = pd.merge(monthly_operating_capacity, supply_input[['Name', 'Capacity']], left_on='Plant', right_on='Name', how='left')
monthly_operating_capacity['% Operating Capacity'] = np.round(100*monthly_operating_capacity['Power Generated (MWh)']/monthly_operating_capacity['Capacity'],2)
monthly_operating_capacity.drop(columns=['Name','Power Generated (MWh)'], inplace=True)

monthly_operation_plan_df = pd.pivot_table(monthly_operation_plan, values='Power Generated (MWh)', index='Plant', columns='Month').reindex(columns=month_names).reset_index()
monthly_operating_capacity_df = pd.pivot_table(monthly_operating_capacity, values='% Operating Capacity', index='Plant', columns='Month').reindex(columns=month_names).reset_index()

with pd.ExcelWriter('PowerGrid_Operation_Plan.xlsx') as writer:
    monthly_operating_capacity_df.to_excel(writer, sheet_name='Operating Capacity', index=False)
    monthly_operation_plan_df.to_excel(writer, sheet_name='Power Generation', index=False)

In [55]:
# Monthly Power Distribution Plan for each Plant

monthly_distribution_plan = pd.DataFrame(columns=['Plant', 'City', 'Month', 'Power Distributed (MWh)'])

for i in range(I):
    for j in range(J):
        for k in range(K):
            if x[i,j,k].x > 0:
                monthly_distribution_plan = pd.concat([monthly_distribution_plan, pd.DataFrame({'Plant': plant_names[i], 'City': city_names[j], 'Month': month_names[k], 'Power Distributed (MWh)': x[i,j,k].x - (x[i,j,k].x*(0.08+0.07*distance_matrix[i][j]))}, index=[0])])

monthly_distribution_plan_t = distance_input[['muni_name','station_name']]
monthly_distribution_plan_t.columns = ['City','Plant']
month_df = pd.DataFrame({'Month': month_names})
monthly_distribution_plan_t['key'] = 1
month_df['key'] = 1
monthly_distribution_plan_t = pd.merge(monthly_distribution_plan_t, month_df, on='key').drop('key', axis=1)
monthly_distribution_plan_t = pd.merge(monthly_distribution_plan_t, monthly_distribution_plan, on=['Plant', 'City', 'Month'], how='left')
monthly_distribution_plan_t['Power Distributed (MWh)'] = monthly_distribution_plan_t['Power Distributed (MWh)'].fillna(0)
monthly_distribution_plan_t['Power Distributed (MWh)'] = np.ceil(monthly_distribution_plan_t['Power Distributed (MWh)'])

monthly_distribution_plan_df = pd.pivot_table(monthly_distribution_plan_t, values='Power Distributed (MWh)', index=['City','Plant'], columns='Month').reindex(columns=month_names).reset_index()
monthly_distribution_plan_df['Total Distribution'] = monthly_distribution_plan_df.loc[:, 'Jan':'Dec'].sum(axis=1)
monthly_distribution_plan_df = monthly_distribution_plan_df.loc[monthly_distribution_plan_df['Total Distribution']>0]
monthly_distribution_plan_df.drop(columns=['Total Distribution'], inplace=True)

# Monthly Demand Fulfilment Matrix

expected_demand_monthly = expected_demand.copy()
month_mapping = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
expected_demand_monthly['Month'] = expected_demand_monthly['Month'].map(month_mapping)
expected_demand_monthly.columns=['City','Month','Demand']

monthly_demand_fulfilment = monthly_distribution_plan_t.copy()
monthly_demand_fulfilment = pd.merge(monthly_demand_fulfilment, expected_demand_monthly, how='left')
monthly_demand_fulfilment['% Fulfilled'] = np.round(100*monthly_demand_fulfilment['Power Distributed (MWh)']/monthly_demand_fulfilment['Demand'],0)

monthly_demand_fulfilment_df = pd.pivot_table(monthly_demand_fulfilment, values='% Fulfilled', index=['City','Plant'], columns='Month').reindex(columns=month_names).reset_index()
monthly_demand_fulfilment_df['Total Demand Fulfilled'] = monthly_demand_fulfilment_df.loc[:, 'Jan':'Dec'].sum(axis=1)
monthly_demand_fulfilment_df = monthly_demand_fulfilment_df.loc[monthly_demand_fulfilment_df['Total Demand Fulfilled']>0]
monthly_demand_fulfilment_df.drop(columns=['Total Demand Fulfilled'], inplace=True)

with pd.ExcelWriter('PowerGrid_Distribution_Plan.xlsx') as writer:
    monthly_demand_fulfilment_df.to_excel(writer, sheet_name='Demand Fulfilment', index=False)
    monthly_distribution_plan_df.to_excel(writer, sheet_name='Distribution Plan', index=False)